In [1]:
#Init Libraries
from openai import OpenAI

#Env Variables
from dotenv import load_dotenv
import os

#LLM
from groq import Groq
from groqllm import GroqLLM

#DML
import pandas as pd

#### Setting up OpenAI GPT4 Mini

In [11]:
token = os.getenv("GITHUB_TOKEN")
route = "https://models.inference.ai.azure.com"
model_name = "gpt-4o-mini"


In [12]:
client = OpenAI(
    base_url=route,
    api_key=token,
)

In [13]:
def get_response(prompt):
    response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": """Instructions:
                        - Be helpful and answer questions concisely. If you don't know the answer, say 'I don't know'
                        - Be concise and to the point.""",
        },
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model=model_name,
    temperature=0.1,
    max_tokens=1024,
    top_p=1.
)
    return response.choices[0].message.content

#### Setting up GroqLLM


In [2]:
llm = GroqLLM(model_name = "llama3-8b-8192"
             ,client =Groq(api_key=os.getenv("GROQ_API_KEY"))
             ,temperature =0.1
             ,system_prompt = ("""
Instructions:
- You are under an examination setting, your goal is to accurately answer the questions to your best ability.
- You will be evaluated after the exam.
- Keep your answers short.
- You will be penalised if you are writing unnessecary words
Question: {query_str}
"""))

#### Loading Data

In [3]:
df_qa = pd.read_csv('../data/MainDataset/data/QA.csv')

In [4]:
question_list = []
for questions in df_qa['question']:
    question_list.append(questions)

In [5]:
import time
response_list = []
for i in range(len(question_list)): 
    time.sleep(5)
    print(question_list[i])
    #response = get_response(question_list[i])
    response = llm.complete(question_list[i])
    print(response)
    rowIndex = df_qa.index[i]
    #response_list.append(response)
    df_qa.at[rowIndex, 'gen_answer'] = response.text
    print('')
    print("--"*100)
    print(f'{i}/{128}')
 


who introduced modern methods of surgery, such as antiseptics, sterilization, and washing hands? brainly
Joseph Lister.

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
0/128
president adams
John Adams was the second President of the United States, serving from 1797 to 1801.

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
1/128
define semitic
Semitic refers to a language family originating from the Middle East, including languages such as Arabic, Hebrew, and Amharic.

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
2/128
what is wudfh

In [8]:
df_qa

,question,answers,gen_answer
0,"who introduced modern methods of surgery, such...",Joseph Lister,"Joseph Lister, a British surgeon, introduced m..."
1,president adams,John Quincy Adams was an American statesman wh...,"John Adams, the 2nd President of the United St..."
2,define semitic,A member of any of a number of peoples of anci...,Semitic refers to a language family originatin...
3,what is wudfhost service,It is a set of Microsoft tools that aid in the...,Wudfhost is a Windows service that manages the...
4,what is an overactive pancreas,An overactive pancrease would produce more enz...,"An overactive pancreas, also known as hyperins..."
...,...,...,...
123,Vice President Agnew describes Civil disobedie...,"['muggers, arsonists, draft evaders, campaign ...",Vice President Spiro Agnew described civil dis...
124,Civil disobedience has been argued in more rec...,"['ambiguity', 'debased', 'ambiguity', 'ambigui...",Civil disobedience has been argued in recent t...
125,What person argues that civil disobedience is ...,"['Marshall Cohen', 'Marshall Cohen', 'Marshall...",Hugo Adam Bedau argues that civil disobedience...
126,"In modern times, what is said about civil diso...","['become utterly debased', 'utterly debased', ...","In modern times, civil disobedience is often v..."


In [6]:
df_qa.to_csv(f'../data/MainDataset/results/Official/llama-8b-8192.csv',index=False)

: 